<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Selenium_NASA_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


     |████████████████████████████████| 911kB 12.0MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://

In [5]:

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from lxml import html
import time

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
class Driver():
    def __init__(self):
        self.driver = webdriver.Chrome(options=options)
    def __enter__(self):
        return self.driver
    def __exit__(self, type, value, traceback):
        self.driver.close()

In [6]:
def crawl(url, f, driver):
    global num_links, processed
    if url in processed:
        return
    processed.add(url)
    num_links+=1
    f.write(f'{num_links} {url}\n')
    f.flush()
    print(f'{num_links} {url}')
    driver.get(root + url)
    time.sleep(1)
    tree = html.fromstring(driver.page_source)
    links=tree.xpath('//a[starts-with(@href,"/") and string-length(@href)>1]/@href')
    for link in links:
        crawl(link, f, driver)

In [ ]:
root = 'https://www.nasa.gov'
processed=set()
num_links=0
with Driver() as driver:
  with open('links.txt', 'w') as f:
      crawl('', f, driver)

In [9]:
links = {'/'}
processed = set()
new_links=set()

with Driver() as driver:
  with open('links.txt', 'w') as f:
      while len(links)>0:
          for i, link in enumerate(links):
              driver.get(root + link)
              time.sleep(1)
              tree = html.fromstring(driver.page_source)
              page_links = tree.xpath('//a[starts-with(@href,"/") and string-length(@href)>1]/@href')
              new_links.update(page_links)
              processed.add(link)
              print(f'{i}/{len(links)}/{len(new_links)} {link}')
              f.write(f'{len(processed)} {link}\n')
              f.flush()
          links = new_links-processed
          print(f'processed: {len(processed)}  links: {len(links)}')

0/1/59 /
processed: 1  links: 59
0/59/65 /multimedia/imagegallery/index.html
1/59/75 /mission_pages/hubble/main/index.html
2/59/185 /missions/
3/59/190 /press-release/nasa-us-european-partner-satellite-returns-first-sea-level-measurements
4/59/193 /stem
5/59/193 /connect/apps.html
6/59/193 /topics/humans-in-space
7/59/193 /nasalive


MaxRetryError: ignored